# Домашнее задание №10
## Логистическая регрессия. Часть 1.
### PROC LOGISTIC

_[Гончаренко Дмитрий Александрович, Жуков Павел Николаевич, Слабоузова Анна Константиновна]_

### 0. Подготовка файлов для выполнения ДЗ.

- НАБОР ДАННЫХ ДЛЯ ЗАДАНИЯ: HW10_DATA.csv (в слайдах на лекции он назывался 'develop', там же можно посмотреть список переменных)

Ответьте на вопросы ниже, не забудьте указать все, на что Вы обратили внимание, отвечая на вопрос, делая соответствующие выводы (график/таблицу/статистику/p-value/прочее).

### Задание


### 1. Предварительное  исследование данных  
#### 1.1 Посмотрите на статистики и распределения переменных.

In [1]:
/* загружаем данные */
proc import file="HW10_DATA.csv"
    out=develop 
    DBMS=CSV
    REPLACE;
run;    

/* Смотрим на статистику переменных в develop*/
title 'Develop statistics';
proc means data=develop;
run;

SAS Connection established. Subprocess id is 4905



Variable,N,Mean,Std Dev,Minimum,Maximum
AcctAge DDA DDABal CashBk Checks DirDep NSF NSFAmt Phone Teller Sav SavBal ATM ATMAmt POS POSAmt CD CDBal IRA IRABal LOC LOCBal ILS ILSBal MM MMBal MMCred MTG MTGBal CC CCBal CCPurc SDB Income HMOwn LORes HMVal Age CRScore Moved InArea Ins Dep DepAmt Inv InvBal,30194 32264 32264 32264 32264 32264 32264 32264 28131 32264 32264 32264 32264 32264 28131 28131 32264 32264 32264 32264 32264 32264 32264 32264 32264 32264 32264 32264 32264 28131 28131 28131 32264 26482 26731 26482 26482 25907 31557 32264 32264 32264 32264 32264 28131 28131,5.9086772 0.8156459 2170.02 0.0159621 4.2599182 0.2955616 0.0870630 2.2905464 0.4056024 1.3652678 0.4668981 3170.60 0.6099368 1235.41 1.0756816 48.9261782 0.1258368 2530.71 0.0532792 617.5704550 0.0633833 1175.22 0.0495909 517.5692344 0.1148959 1875.76 0.0563786 0.0493429 8081.74 0.4830969 9586.55 0.1541716 0.1086660 40.5889283 0.5418802 7.0056642 110.9121290 47.9283205 666.4935197 0.0296305 0.9602963 0.3463613 2.1346082 2232.76 0.0296826 1599.17,6.5228043 0.3877787 7282.73 0.1313679 5.1566429 0.4563019 0.2819317 14.3636766 1.1994796 2.2837649 0.4989108 13397.15 0.4877718 4462.28 2.8886792 137.0504390 0.3316705 14811.78 0.2245932 7516.56 0.2436549 9605.28 0.2171015 2285.19 0.3189012 5484.48 0.2874843 0.2165864 79794.12 0.4997231 81383.95 0.4473868 0.3112245 28.5149786 0.4982523 2.8380147 21.9499213 14.1108480 40.4132381 0.1695685 0.1952652 0.4758174 1.7217969 6704.29 0.1697131 56940.13,0.3000000 0 -774.8300000 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -613.0000000 0 0 0 0 0 0 0 0 -2060.51 0 0 0 0 0.5000000 67.0000000 16.0000000 509.0000000 0 0 0 0 0 0 -2214.92,61.5000000 1.0000000 278093.83 4.0000000 49.0000000 1.0000000 1.0000000 666.8500000 30.0000000 27.0000000 1.0000000 700026.94 1.0000000 427731.26 54.0000000 3293.49 1.0000000 1053900.00 1.0000000 596497.60 1.0000000 523147.24 1.0000000 29162.79 1.0000000 120801.11 5.0000000 1.0000000 10887573.28 1.0000000 10641354.78 5.0000000 1.0000000 233.0000000 1.0000000 19.5000000 754.0000000 94.0000000 820.0000000 1.0000000 1.0000000 1.0000000 28.0000000 484893.67 1.0000000 8323796.02


In [2]:
/* Находим количество всех наблюдений.
Используется, чтобы определить по выводу процедуры MEANS какие значения отсутствуют*/
data _null_;
    set develop nobs=number_obs;
    put "NUMBER OF ALL OBSERVATIONS == " number_obs;
    stop;
run;

In [ ]:
/* Сокращаем количество переменных до тех,
которые были указаны на лекции*/

data develop;
    set develop;
    keep ins dda ddabal dep depamt cashbk checks res;
run;

title 'Short develop';
proc print data=develop(obs=5); run;

#### Q1.1: Укажите все переменные, в которых встречаются пропущенные значения:
AcctAge, Phone, POS, POSAmt, CC, CCBal, CCPurc, Income, HMOwn, LORes, HMVal, Age, CRScore, Inv, InvBal

#### 1.2 Разделите набор на две части: train и test. 
Далее всю разработку мы проводим на части train, а к test мы применяем полученный результат. См. ex_train_val.sas

In [4]:
%include 'ex_TRAIN_VAL.sas'

In [5]:
data train test;
    set develop;
    if Selected=1 then output train;
    else output test;
    drop Selected SelectionProb SamplingWeight;
run;

/* title 'Statistics for train';
proc means data=train; run;
title 'Statistics for test';
proc means data=test; run; */

#### Q1.2: Можно ли утверждать, что распределение целевой переменной в частях train и test почти одинаковое? 
Можно, так как наборы формировались таким образом, что у них равное соотношение данных с различным ins.
То есть если посмотреть на вывод процедур means для train и test, то можно увидеть поле mean для ins практически одинаковое.

#### Q1.3: Какой оператор контролирует похожесть распределения? 
strata ins в процедуре SURVEYSELECT 

### 2. Выбор моделей кандидатов

#### 2.1 Выберите 2 модели, используя метод all subsets regression (и Хи-квадрат как метрику качества).

In [10]:
title 'All subsets regression';
proc logistic data=train;
    model ins(event='1') = AcctAge DDA DDABal CashBk Checks DirDep 
                        NSF NSFAmt Phone Teller Sav SavBal ATM ATMAmt
                        POS POSAmt CD CDBal IRA IRABal LOC LOCBal ILS
                        ILSBal MM MMBal MMCred MTG MTGBal CC CCBal CCPurc
                        SDB Income HMOwn LORes HMVal Age CRScore Moved InArea
                        Dep DepAmt Inv InvBal / selection=SCORE
        best = 3
        start= 1
        stop = 7;
    ods output bestsubsets=work.score responseprofile=work.profile;
run;

#### Q2.1 Почему вы выбрали их?
Выбраны модели с максимальным значением хи-квадрат для модели с 6 параметрами. 
Количество 6 - показалось оптимальным ввиду того, для 7 параметров значение хи-квадрат практически такое же как для 6. То есть особого выиграша 7-ой параметр не приносит. 
#### Q2.2 Какие переменные вошли в модели, которые вы выбрали?
В первую модель: `DDA DDABal Sav SavBal CD MM`.
Во вторую модель: `DDA DDABal Sav SavBal CD MMBal`.

#### 2.2 Исследование выбранных моделей.

 - Постройте выбранные модели (т.е. оцените коэффициенты при переменных). 
 - Примените их к набору test. 
 - Постройте K-S (`PROC NPAR1WAY`) и ROC (`PROC LOGISTIC`) на наборах train и test. Оцените «устойчивость» модели. (см. ex_KS.sas и ex_ROC_comp.sas)


In [30]:
proc logistic data=train noprint;
    model ins(event='1')=DDA DDABal Sav SavBal CD MM / selection=stepwise;
    code file='logreg_scorecode_m1.sas';
run;
data Develop_Scored_Model1;
    set test;
    %include 'logreg_scorecode_m1.sas';
    keep ins P_Ins1;
run;
proc print data=Develop_Scored_Model1(obs=5); run;

Obs,Ins,P_Ins1
1,0,0.25693
2,0,0.16942
3,0,0.46518
4,0,0.18114
5,0,0.28325


In [29]:
proc logistic data=train ;
    model ins(event='1')=DDA DDABal Sav SavBal CD MMBal / selection=stepwise;
    code file='logreg_scorecode_m2.sas';
run;
data Develop_Scored_Model2;
    set test;
    %include 'logreg_scorecode_m2.sas';
    keep ins P_Ins1;
run;
proc print data=Develop_Scored_Model2(obs=5); run;

#### Q2.3 Все ли коэффициенты в обеих моделях значимые?
Да. Определялось по полям `Pr>ChiSq` напротив каждой переменной.
#### Q2.4 Модели устойчивы? Как вы пришли к такому выводу? Какие шаги вы бы предложили предпринять для дальнейшего повышения устойчивости моделей?
... 
#### Q2.5 Чему равен индекс Джини для обеих моделей на train и test?
... 